In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import OOPAO
import tomoAO
from importlib import reload


In [ ]:
ao_mode = "MLAO"

config_dir = "/home/joaomonteiro/Desktop/OOPAO_ast/tutorials/"
config_file = "config.ini"

# Loading the config
config_vars = tomoAO.IO.load_from_ini(config_file, ao_mode=ao_mode,config_dir=config_dir)

In [ ]:
from OOPAO.Source import Source
from OOPAO.Asterism import Asterism
reload(OOPAO.Source)
reload(OOPAO.Asterism)
from OOPAO.Source import Source
from OOPAO.Asterism import Asterism


optBand = config_vars["lgs_opticalBand"]
magnitude = config_vars["lgs_magnitude"]
lgs_zenith = config_vars["lgs_zenith"]
lgs_azimuth = config_vars["lgs_azimuth"]

n_lgs = 4

lgsAst = Asterism([Source(optBand=optBand,
              magnitude=magnitude,
              coordinates=[lgs_zenith[kLgs], lgs_azimuth[kLgs]])
          for kLgs in range(n_lgs)])


In [ ]:
SciSrc = Source('K',10)


In [ ]:
from OOPAO.Telescope import Telescope
reload(OOPAO.Telescope)
from OOPAO.Telescope import Telescope


sensing_wavelength = lgsAst.src[0].wavelength      # sensing wavelength of the WFS, read from the ngs object
n_subaperture      = 20                  # number of subaperture across the diameter
diameter           = 8                   # diameter of the support of the phase screens in [m]
resolution         = n_subaperture*8     # resolution of the phase screens in pixels
pixel_size         = diameter/resolution # size of the pixels in [m]
obs_ratio          = 0.1                 # central obstruction in fraction of the telescope diameter
sampling_time      = 1/1000              # sampling time of the AO loop in [s]
fieldOfViewInArcsec = 5

# initialize the telescope object
tel = Telescope(diameter          = diameter,
               resolution         = resolution,
               centralObstruction = obs_ratio,
               samplingTime       = sampling_time,
               fov                = fieldOfViewInArcsec)


In [ ]:
print("lgsAst:")
lgsAst.print_optical_path()

print("\nSciSrc:")
SciSrc.print_optical_path()

In [ ]:
plt.imshow(lgsAst.OPD[0])

In [ ]:
from OOPAO.Atmosphere import Atmosphere
reload(OOPAO.Atmosphere)
from OOPAO.Atmosphere import Atmosphere


r0 = config_vars["r0"]
L0 = config_vars["L0"]

fractionnalR0 = config_vars["fractionnalR0"]
windSpeed = config_vars["windSpeed"]
windDirection = config_vars["windDirection"]
altitude = config_vars["altitude"]


lgsAst**tel


atm = Atmosphere(telescope      = tel,
                 r0             = r0,
                 L0             = L0,
                 fractionalR0   = fractionnalR0,
                 altitude       = altitude,
                 windDirection  = windDirection,
                 windSpeed      = windSpeed)


atm.initializeAtmosphere(telescope=tel)


In [ ]:
lgsAst.print_optical_path()

In [ ]:
lgsAst**tel

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 4))

for i in range(n_lgs):
    axes[i].imshow(lgsAst.OPD[i])
    axes[i].axis('off')
    axes[i].set_title(f'Asterism OPD {i}')


In [ ]:
tel+atm

In [ ]:
for n in range(1000):
    lgsAst**tel

    if n%100 == 0:
        fig, axes = plt.subplots(1, 4, figsize=(20, 4))

        for i in range(n_lgs):
            axes[i].imshow(lgsAst.OPD[i])
            axes[i].axis('off')
            axes[i].set_title(f'Asterism OPD {i}')

        plt.show()
    atm.update()


In [ ]:
lgsAst**tel


In [ ]:
print("lgsAst:")
lgsAst.print_optical_path()

print("\nSciSrc:")
SciSrc.print_optical_path()

In [ ]:
plt.imshow(lgsAst.OPD[0])

In [ ]:
SciSrc**tel

In [ ]:
print("lgsAst:")
lgsAst.print_optical_path()

print("\nSciSrc:")
SciSrc.print_optical_path()

In [ ]:
atm.asterism

In [ ]:
atm.telescope.src.coordinates

In [ ]:
SciSrc**tel

In [ ]:
tel-atm

In [ ]:
tel+atm

In [ ]:
atm.telescope.src.coordinates

In [ ]:
plt.imshow(SciSrc.OPD)
plt.colorbar()

In [ ]:
atm.telescope.src.coordinates

In [ ]:
atm.asterism

In [ ]:
lgsAst**tel

In [ ]:
plt.imshow(lgsAst.OPD[1]-SciSrc.OPD)
plt.colorbar()

In [53]:
import numpy as np
import math
import numba as nb
from scipy.special import gamma
from scipy.sparse import block_diag

@nb.njit(nb.complex128(nb.complex128), cache=False)
def _kv56_scalar(z):
    """Scalar implementation used as kernel for array version"""
    # Precomputed Gamma function values for v=5/6
    gamma_1_6 = 5.56631600178  # Gamma(1/6)
    gamma_11_6 = 0.94065585824  # Gamma(11/6)
    # Precompute constants for numerical stability
    # Constants for the series expansion and asymptotic approximation
    v = 5.0 / 6.0
    z_abs = np.abs(z)
    if z_abs < 2.0:
        # Series expansion for small |z|
        sum_a = 0.0j
        sum_b = 0.0j
        term_a = (0.5 * z)**v / gamma_11_6
        term_b = (0.5 * z)**-v / gamma_1_6
        sum_a += term_a
        sum_b += term_b
        z_sq_over_4 = (0.5 * z)**2
        k = 1
        tol = 1e-15
        max_iter = 1000
        for _ in range(max_iter):
            factor_a = z_sq_over_4 / (k * (k + v))
            term_a *= factor_a
            sum_a += term_a
            factor_b = z_sq_over_4 / (k * (k - v))
            term_b *= factor_b
            sum_b += term_b
            if abs(term_a) < tol * abs(sum_a) and abs(term_b) < tol * abs(sum_b):
                break
            k += 1
        K = np.pi * (sum_b - sum_a)
    else:
        # Asymptotic expansion for large |z|
        z_inv = 1.0 / z
        sum_terms = 1.0 + (2.0/9.0)*z_inv + (-7.0/81.0)*z_inv**2 + \
                    (175.0/2187.0)*z_inv**3 + (-2275.0/19683.0)*z_inv**4 + \
                    (5005.0/177147.0)*z_inv**5  #+ (-2662660.0/4782969.0)*z_inv**6
        prefactor = np.sqrt(np.pi/(2.0*z)) * np.exp(-z)
        K = prefactor * sum_terms
    return K

# Vectorized version outside the class
@nb.vectorize([nb.complex128(nb.complex128),  # Complex input
            nb.complex128(nb.float64)],    # Real input
            nopython=True, target='parallel')
def _kv56(z):
    """
    Modified Bessel function K_{5/6}(z) for numpy arrays
    Handles both real and complex inputs efficiently
    """
    return _kv56_scalar(z)




In [68]:
from numba import njit, prange
from math import gamma

@njit(parallel=True)
def bessel_i_series_numba(x, n, terms=50):
    size = x.shape[0]
    result = np.zeros(size, dtype=np.float64)

    for idx in prange(size):
        xi = x[idx]
        coeff = (xi / 2.0) ** n / gamma(n + 1)
        term = coeff
        sum_result = term
        x_sq_half = (xi / 2.0) ** 2

        for m in range(1, terms):
            term *= x_sq_half / (m * (m + n))
            sum_result += term

        result[idx] = sum_result

    return result


# Vectorized version outside the class
@nb.vectorize([nb.complex128(nb.complex128),  # Complex input
            nb.complex128(nb.float64)],    # Real input
            nopython=True, target='parallel')
def _kv56(z):
    """
    Modified Bessel function K_{5/6}(z) for numpy arrays
    Handles both real and complex inputs efficiently
    """


    return (np.pi / 2) * (bessel_i_series_numba(5/6, z) - bessel_i_series_numba(-5/6, z)) / np.sin(5/6 * np.pi)



TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'shape' of type float64

File "../../../../../tmp/ipykernel_16049/834780515.py", line 6:
<source missing, REPL/exec in use?>

During: typing of get attribute at /tmp/ipykernel_16049/834780515.py (6)

File "../../../../../tmp/ipykernel_16049/834780515.py", line 6:
<source missing, REPL/exec in use?>

During: Pass nopython_type_inference
During: resolving callee type: type(CPUDispatcher(<function bessel_i_series_numba at 0x7409ff3672e0>))
During: typing of call at /tmp/ipykernel_16049/834780515.py (36)

During: resolving callee type: type(CPUDispatcher(<function bessel_i_series_numba at 0x7409ff3672e0>))
During: typing of call at /tmp/ipykernel_16049/834780515.py (36)

During: resolving callee type: type(CPUDispatcher(<function bessel_i_series_numba at 0x7409ff3672e0>))
During: typing of call at /tmp/ipykernel_16049/834780515.py (36)


File 

In [62]:
u = np.random.rand(10000000)

In [65]:
sp = kv(5/6, u)
sp

array([ 0.86689741,  0.69178212,  2.77652309, ...,  1.4163619 ,
        3.29191202, 10.03222968], shape=(10000000,))

In [66]:
pt = np.real(_kv56(u.astype(np.complex128)))
pt


array([ 0.86689741,  0.69178212,  2.77652309, ...,  1.4163619 ,
        3.29191202, 10.03222968], shape=(10000000,))